# ResNet-50 Model

This notebook will define the ResNet-50 Model architecture we will be using, adapted to the LesionDataset class definition. 

Common errors when running include: deprecated packages or python version (3.10.10+ recommended), incompatible notebook kernel (if persistent, try setting it to the global kernel), missing requirements, not setting the dataset path to your local path

In [ ]:
import sys

sys.path.insert(0, "..")


Importing dataset and printing to verify path correctness

In [ ]:
from dataset import LesionDataset

dataset = LesionDataset('data') #local path to data
for patient in dataset.patients():
    """Patient is an instance of the Patient class

    The `Patient` class exposes a number of methods for accessing a patient's associated data:
    - `Patient.image_data(image_type)` - returns the image, prostate mask, and lesion arrays
    corresponding to `image_type`
    - `Patient.world_matrix()` - returns the world matrix of `image_type`
    - `Patient.spacing(image_type)` - returns the voxel spacing of `image_type`
    - `Patient.bvals()` - returns the b-values of the DWI images
    - `Patient.slice_data(image_type, slice_index, crop_to_mask=False, pad=0)` - returns a particular
    slice of the image/mask data, optionally cropped to the prostate mask for easier visualization.
    """
    print(patient.patient_id)

print(dataset.query('ProstateX-0000'))

Setting up ResNet-50 Architecture

In [ ]:
# start setting up classes for resnet50

import torch
import torch.nn as nn
import torch.nn.functional as F

class ResNet50(nn.Module):
    def __init__(self, num_classes=1000):
        super(ResNet50, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 64, 3)
        self.layer2 = self._make_layer(256, 128, 4, stride=2)
        self.layer3 = self._make_layer(512, 256, 6, stride=2)
        self.layer4 = self._make_layer(1024, 512, 3, stride=2)

        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(2048, num_classes)


        self._initialize_weights()

    def _make_layer(self, in_planes, out_planes, blocks, stride=1):
        layers = []
        layers.append(nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False))
        layers.append(nn.BatchNorm2d(out_planes))
        layers.append(nn.ReLU(inplace=True))
        for i in range(1, blocks):
            layers.append(nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False))
            layers.append(nn.BatchNorm2d(out_planes))
            layers.append(nn.ReLU(inplace=True))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

model = ResNet50()
print(model)

Sample image flattening example

In [ ]:
import numpy as np
img1 = np.load('data/ProstateX-0000/images/ADC.npy')
# print(img1.shape)
img2 = np.load('data/ProstateX-0000/images/CDIs.npy')
# print(img2.shape)
img3 = np.load('data/ProstateX-0000/images/DWI.npy')
# print(img3.shape)

# Img1 shape: (128, 84, 19)
# Img2 shape: (128, 84, 19)
# Img3 shape: (3, 128, 84, 19)

img3_new = img3[0, :, :, :]
print(img3_new.shape)
img_stacked = np.stack((img1, img2, img3_new), axis=3)
print(img_stacked.shape)
img_stacked = np.reshape(img_stacked, (128, 84, 57))
print(img_stacked.shape)

# img_stacked shape: (128, 84, 57)


